In [1]:
import os
import json
import sys
import great_expectations as ge
import great_expectations.jupyter_ux
from datetime import datetime
import math
import pandas as pd
os.chdir('/Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations')

import ge_prod.ge_data_access as gda
from ge_prod.queries import queries

2020-02-26T14:18:45-0500 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.


# Author Expectations



[**Watch a short tutorial video**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#video)

[**Read more in the tutorial**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations)

**Reach out for help on** [**Great Expectations Slack**](https://tinyurl.com/great-expectations-slack)


### Get a DataContext object
[Read more in the tutorial](https://great-expectations.readthedocs.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-datacontext-object)




In [2]:
context = ge.data_context.DataContext()

2020-02-26T14:18:47-0500 - INFO - Using project config: /Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations/great_expectations.yml


### List data assets in your project

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#data-assets)


In [3]:
great_expectations.jupyter_ux.list_available_data_asset_names(context)

Inspecting your data sources. This may take a moment...


#### Pick one of the data asset names above and use as the value of data_asset_name argument below

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-batch)


### Specify data_asset & expectation_suite_name

In [4]:
data_asset_name = 'network_outreach'
normalized_data_asset_name = context.normalize_data_asset_name(data_asset_name)
print(normalized_data_asset_name)

NormalizedDataAssetName(datasource='agero_dsa_pandas', generator='default', generator_asset='network_outreach')


### Create a new empty expectation suite

In [5]:
expectation_suite_name = 'warnings_dec2019'
# context.create_expectation_suite(data_asset_name=data_asset_name, expectation_suite_name=expectation_suite_name,
#                                 overwrite_existing=False)

### Get batch to create expectations against

In [6]:
query = queries.get('network_outreach').get('create_expectations_dec2019')

In [7]:
query

"SELECT * FROM network_outreach WHERE dispatch_request_time_utc >= to_date('2019-12-01') and dispatch_request_time_utc <= to_date('2019-12-31');"

In [8]:
rule_df = gda.snowflake_connector_to_df(query)
batch_kwargs = {'dataset': rule_df}

In [9]:
batch = context.get_batch(normalized_data_asset_name, expectation_suite_name=expectation_suite_name,
                         batch_kwargs=batch_kwargs)
batch.head()

,CASE_ID,TASK_ID,VENDOR_ID,VENDOR_ADDRESS_ID,VENDOR_CATEGORY,VENDOR_CANDIDATE_NUMBER,DISPATCH_RESPONSE,DISPATCH_RESPONSE_REASON,DISPATCH_TYPE,ENROUTE_MILES,...,VENDOR_PHONE,VENDOR_FAX,ETA_MINUTES,TOTAL_ETA_EXTENSION_MINUTES,TOTAL_ETA_EXTENSION_COUNT,IS_DISPATCHED_VENDOR,SERVICE_TIME_EASTERN,SERVICE_TIME_UTC,DISPATCH_REQUEST_TIME_EASTERN,DISPATCH_REQUEST_TIME_UTC
0,2295319688,1,52998,1,D,3,REFUSED,Do not have the proper equipment at this time,dispatch_bot,34.71,...,6087845872,6087840727,NaN,0.0,0,False,2019-12-30 12:30:55,2019-12-30 17:30:55,2019-12-30 12:34:14,2019-12-30 17:34:14
1,2295282365,1,113002,1,D,9,ACCEPTED,Available,agent_call,33.35,...,8129239920,8129232820,90.0,0.0,0,True,2019-12-30 01:41:23,2019-12-30 06:41:23,2019-12-30 01:40:00,2019-12-30 06:40:00
2,2295324153,1,24485,1,PA,1,REFUSED,Out of My Coverage Area,agent_call,4.64,...,8437622421,8437622422,NaN,0.0,0,False,2019-12-30 13:54:20,2019-12-30 18:54:20,2019-12-30 13:54:24,2019-12-30 18:54:24
3,2295336508,1,100519,1,PA,1,ACCEPTED,Available,edispatch,20.49,...,7027249000,7027249007,45.0,0.0,0,True,2019-12-30 14:23:27,2019-12-30 19:23:27,2019-12-30 14:23:31,2019-12-30 19:23:31
4,2295290956,1,123670,3,D,4,REFUSED,Out of My Coverage Area,agent_call,36.56,...,8106679400,None,NaN,0.0,0,False,2019-12-30 08:55:32,2019-12-30 13:55:32,2019-12-30 08:55:32,2019-12-30 13:55:32


In [10]:
type(batch)

great_expectations.dataset.pandas_dataset.PandasDataset

In [11]:
print(batch.get_row_count())

1991435


In [12]:
print(rule_df.shape)

(1991435, 24)


In [13]:
# getting rule_df batchId & fingerprint
rule_batch_fingerprint = batch.batch_fingerprint
rule_batch_id = batch.batch_id

In [14]:
print('rule_batch_fingerprint: ', '\n', rule_batch_fingerprint)
print('rule_batch_id: ', '\n', rule_batch_id)

rule_batch_fingerprint:  
 {'partition_id': '20200226T191946.738015Z', 'fingerprint': 'e99420d7b7accce7c3d15c96ac6d68c5'}
rule_batch_id:  
 {'timestamp': 1582744784.9986188, 'PandasInMemoryDF': True, 'fingerprint': 'ecc34d0d3b9f984866a2e747353f9dd8'}


## Author Expectations

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#create-expectations)

See available expectations in the [expectation glossary](https://docs.greatexpectations.io/en/latest/glossary.html?utm_source=notebook&utm_medium=create_expectations)


### Dataset exploration & understanding of fields to ensure rules reflect behavior of data
- Validating columns to exist in table shape
- Expected column count in table shape
- Expected set values to be seen in given column
- Expected columns to have null or non-null values X percentage of the time
- Expect column values to be of certain data type(s)
- Placing max and min values limits on numerical columns
- Average or median column value to be within a certain range
- Expecting column A to be large/less than column B

### 1. Validating to see if every column exists in table

In [15]:
# add more expectations here
column_names = batch.get_table_columns()
print(column_names)

['CASE_ID', 'TASK_ID', 'VENDOR_ID', 'VENDOR_ADDRESS_ID', 'VENDOR_CATEGORY', 'VENDOR_CANDIDATE_NUMBER', 'DISPATCH_RESPONSE', 'DISPATCH_RESPONSE_REASON', 'DISPATCH_TYPE', 'ENROUTE_MILES', 'VENDOR_NAME', 'VENDOR_ADDRESS_1', 'VENDOR_ADDRESS_2', 'VENDOR_ZIP', 'VENDOR_PHONE', 'VENDOR_FAX', 'ETA_MINUTES', 'TOTAL_ETA_EXTENSION_MINUTES', 'TOTAL_ETA_EXTENSION_COUNT', 'IS_DISPATCHED_VENDOR', 'SERVICE_TIME_EASTERN', 'SERVICE_TIME_UTC', 'DISPATCH_REQUEST_TIME_EASTERN', 'DISPATCH_REQUEST_TIME_UTC']


In [16]:
# add more expectations here
for col in column_names:
    print(col + ':',batch.expect_column_to_exist(col))

CASE_ID: {'success': True}
TASK_ID: {'success': True}
VENDOR_ID: {'success': True}
VENDOR_ADDRESS_ID: {'success': True}
VENDOR_CATEGORY: {'success': True}
VENDOR_CANDIDATE_NUMBER: {'success': True}
DISPATCH_RESPONSE: {'success': True}
DISPATCH_RESPONSE_REASON: {'success': True}
DISPATCH_TYPE: {'success': True}
ENROUTE_MILES: {'success': True}
VENDOR_NAME: {'success': True}
VENDOR_ADDRESS_1: {'success': True}
VENDOR_ADDRESS_2: {'success': True}
VENDOR_ZIP: {'success': True}
VENDOR_PHONE: {'success': True}
VENDOR_FAX: {'success': True}
ETA_MINUTES: {'success': True}
TOTAL_ETA_EXTENSION_MINUTES: {'success': True}
TOTAL_ETA_EXTENSION_COUNT: {'success': True}
IS_DISPATCHED_VENDOR: {'success': True}
SERVICE_TIME_EASTERN: {'success': True}
SERVICE_TIME_UTC: {'success': True}
DISPATCH_REQUEST_TIME_EASTERN: {'success': True}
DISPATCH_REQUEST_TIME_UTC: {'success': True}


### 2. Validating column count in table is always the same

In [17]:
print('# of columns in network_outreach: ', len(column_names)), 

# of columns in network_outreach:  24


(None,)

In [18]:
master_col_names = [
		'CASE_ID', 'TASK_ID', 'VENDOR_ID', 'VENDOR_ADDRESS_ID', 'VENDOR_CATEGORY',
		'VENDOR_CANDIDATE_NUMBER', 'DISPATCH_RESPONSE', 'DISPATCH_RESPONSE_REASON', 'DISPATCH_TYPE',
		'ENROUTE_MILES', 'SERVICE_TIME_EASTERN', 'SERVICE_TIME_UTC', 'DISPATCH_REQUEST_TIME_EASTERN',
		'DISPATCH_REQUEST_TIME_UTC', 'VENDOR_NAME', 'VENDOR_ADDRESS_1', 'VENDOR_ADDRESS_2',
		'VENDOR_ZIP', 'VENDOR_PHONE', 'VENDOR_FAX', 'ETA_MINUTES', 'TOTAL_ETA_EXTENSION_MINUTES',
		'TOTAL_ETA_EXTENSION_COUNT', 'IS_DISPATCHED_VENDOR'
	]

In [19]:
len(master_col_names)

24

In [ ]:
if len(column_names) == 18:
    print(batch.expect_table_column_count_to_equal(18, result_format='SUMMARY'))
elif len(column_names) == 19:
    print(batch.expect_table_column_count_to_equal(19, result_format='SUMMARY'))

In [ ]:
# Looking at how often eta with 0 values occurs in the data

print('eta values with 0 values: ', 
      rule_df[rule_df.eta == 0].shape[0] / rule_df.shape[0], 'percent of the time')

In [ ]:
# looking at how often eta is missing in the data

print('null eta values missing: ', 
      rule_df[rule_df.eta.isnull()].shape[0] / rule_df.shape[0], 'percent of the time')

### 3. Checking which columns should not have null values

In [21]:
# identifying which columns should not be null
print(column_names)

['case_id', 'task_id', 'vendor_id', 'vendor_address_id', 'eta', 'eta_extension', 'vendor_category', 'vendor_candidate_number', 'dispatch_response_description', 'dispatch_response_id', 'dispatch_response_reason', 'dispatch_type', 'task_status_code', 'enroute_miles', 'is_dispatch', 'service_time_eastern', 'service_time_utc', 'dispatch_request_time_eastern', 'dispatch_request_time_utc']


In [22]:
rule_df.isnull().sum()

case_id                               0
task_id                               0
vendor_id                             0
vendor_address_id                     0
eta                              775689
eta_extension                         0
vendor_category                   24130
vendor_candidate_number               0
dispatch_response_description         0
dispatch_response_id                  0
dispatch_response_reason              0
dispatch_type                         0
task_status_code                      0
enroute_miles                    100865
is_dispatch                           0
service_time_eastern                  0
service_time_utc                      0
dispatch_request_time_eastern         0
dispatch_request_time_utc             0
dtype: int64

In [23]:
# Separating null & non-null columns
null_cols = list(rule_df.isnull().sum()[rule_df.isnull().sum() > 0].keys())
not_null_cols = list(rule_df.isnull().sum()[rule_df.isnull().sum() == 0].keys())

In [24]:
# checking for all columns that shouldn't be null are not
for col in not_null_cols:
    print(col, '\n', batch.expect_column_values_to_not_be_null(col, result_format='BASIC'))

case_id 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
task_id 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
vendor_id 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
vendor_address_id 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
eta_extension 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
vendor_candidate_number 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
dispatch_response_description 
 {'success': True, 'result': {'element_count': 220207

### 4. Validating columns to have null values
- **columns to check:**
    - eta, vendor_category, and enroute_miles

In [25]:
# grabbing percentage of values that are null by column
null_percents = (rule_df.isnull().sum() / len(rule_df))[rule_df.isnull().sum() / len(rule_df) > 0]
print(null_percents)
print('---------------------------------------')
for x, y in null_percents.iteritems():
    print(x, round(y,3))

eta                0.352254
vendor_category    0.010958
enroute_miles      0.045805
dtype: float64
---------------------------------------
eta 0.352
vendor_category 0.011
enroute_miles 0.046


In [26]:
for weight in null_percents.values:
    print(round(weight,2))

0.35
0.01
0.05


In [48]:
rule_df.head()

,CASE_ID,TASK_ID,VENDOR_ID,VENDOR_ADDRESS_ID,VENDOR_CATEGORY,VENDOR_CANDIDATE_NUMBER,DISPATCH_RESPONSE,DISPATCH_RESPONSE_REASON,DISPATCH_TYPE,ENROUTE_MILES,...,VENDOR_PHONE,VENDOR_FAX,ETA_MINUTES,TOTAL_ETA_EXTENSION_MINUTES,TOTAL_ETA_EXTENSION_COUNT,IS_DISPATCHED_VENDOR,SERVICE_TIME_EASTERN,SERVICE_TIME_UTC,DISPATCH_REQUEST_TIME_EASTERN,DISPATCH_REQUEST_TIME_UTC
0,2295319688,1,52998,1,D,3,REFUSED,Do not have the proper equipment at this time,dispatch_bot,34.71,...,6087845872,6087840727,NaN,0.0,0,False,2019-12-30 12:30:55,2019-12-30 17:30:55,2019-12-30 12:34:14,2019-12-30 17:34:14
1,2295282365,1,113002,1,D,9,ACCEPTED,Available,agent_call,33.35,...,8129239920,8129232820,90.0,0.0,0,True,2019-12-30 01:41:23,2019-12-30 06:41:23,2019-12-30 01:40:00,2019-12-30 06:40:00
2,2295324153,1,24485,1,PA,1,REFUSED,Out of My Coverage Area,agent_call,4.64,...,8437622421,8437622422,NaN,0.0,0,False,2019-12-30 13:54:20,2019-12-30 18:54:20,2019-12-30 13:54:24,2019-12-30 18:54:24
3,2295336508,1,100519,1,PA,1,ACCEPTED,Available,edispatch,20.49,...,7027249000,7027249007,45.0,0.0,0,True,2019-12-30 14:23:27,2019-12-30 19:23:27,2019-12-30 14:23:31,2019-12-30 19:23:31
4,2295290956,1,123670,3,D,4,REFUSED,Out of My Coverage Area,agent_call,36.56,...,8106679400,None,NaN,0.0,0,False,2019-12-30 08:55:32,2019-12-30 13:55:32,2019-12-30 08:55:32,2019-12-30 13:55:32


In [ ]:
# df.notnull().mean()
# df.groupby("Team").apply(lambda x: x.notnull().mean())
# df.set_index("Team").notnull().groupby(level=0).mean()

In [ ]:
df.groupby("DISPATCH_REQUEST_TIME_UTC").apply(lambda x: x.notnull().mean())

In [15]:
df = rule_df.copy()

In [45]:
from typing import Union
from great_expectations.dataset import PandasDataset

In [211]:
def get_df_not_null_weights(df: Union[pd.DataFrame, PandasDataset], date_col: str, not_null_col: str) -> float:
    df_group = df.groupby(df[date_col].dt.day)
    df_group = df_group.apply(lambda x: x[not_null_col].notnull().mean())
    
    original_weight = df_group.mean()
    new_weight = round((original_weight - 0.05), 3)
    if new_weight < 0:
        final_weight = original_weight
    else:
        final_weight = new_weight
    return float(final_weight)

In [212]:
null_cols = ['VENDOR_CATEGORY', 'ENROUTE_MILES']

In [213]:
w = get_df_not_null_weights(df, 'DISPATCH_REQUEST_TIME_UTC', 'ENROUTE_MILES')

In [217]:
weight_cols = ['ETA_MINUTES', 'VENDOR_CATEGORY', 'ENROUTE_MILES']

In [219]:
for col in weight_cols:
    s = get_df_not_null_weights(df, 'DISPATCH_REQUEST_TIME_UTC', col)
    print(col, s)

ETA_MINUTES 0.547
VENDOR_CATEGORY 0.939
ENROUTE_MILES 0.904


In [182]:
g = df.groupby(df.DISPATCH_REQUEST_TIME_UTC.dt.day)
g= g.apply(lambda x: x['VENDOR_CATEGORY'].notnull().mean())

In [187]:
print(g.min().round(3))
print(round(g.min(), 3))

0.979
0.979


In [20]:
# calculating weight for columns of how often they should be null
null_percents = (1 - (batch.isnull().sum() / len(batch))[batch.isnull().sum() / len(batch) > 0])
for x, y in null_percents.items():
    print(x, y, sep='\n')
    
not_null_weights = dict(null_percents)

# lowering weights by one thousandth of decimal
for key, original_weight in not_null_weights.items():
    not_null_weights[key] = round((original_weight - 0.05), 3)
    
print('---------------------------------------')
print('not null weights:')
for x, y in not_null_weights.items():
    print(x,y)

VENDOR_CATEGORY
0.9877234255700036
ENROUTE_MILES
0.9531393191341921
VENDOR_ADDRESS_1
0.9997057398308256
VENDOR_ADDRESS_2
0.006691657021193231
VENDOR_ZIP
0.9997057398308256
VENDOR_PHONE
0.9840748003324236
VENDOR_FAX
0.6389648670431121
ETA_MINUTES
0.5999472742017691
---------------------------------------
not null weights:
VENDOR_CATEGORY 0.938
ENROUTE_MILES 0.903
VENDOR_ADDRESS_1 0.95
VENDOR_ADDRESS_2 -0.043
VENDOR_ZIP 0.95
VENDOR_PHONE 0.934
VENDOR_FAX 0.589
ETA_MINUTES 0.55


In [27]:
# adjusting weights to mostly at this minimum
weights = {'eta': 0.3, 'vendor_category': 0.01, 'enroute_miles': 0.04}

In [28]:
# validating columns that should be null with 
for col, weight in weights.items():
    print(col, '\n', batch.expect_column_values_to_be_null(col, mostly=weight, include_config=True,
                                                           catch_exceptions=True,
                                                           result_format='BASIC'))

eta 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 1426386, 'unexpected_percent': 64.77463301658662, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_null', 'kwargs': {'column': 'eta', 'mostly': 0.3, 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
vendor_category 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 2177945, 'unexpected_percent': 98.90421534234756, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_null', 'kwargs': {'column': 'vendor_category', 'mostly': 0.01, 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
enroute_miles 
 {'success': True, 'result': {'element_count': 2202075, 'unexpected_count': 2101210, 'unexpected_percent': 95.41954747227047, 'par

### 5. Expecting column values to be in a set
- vendor_category
- dispatch_response_description
- dispatch_type
- task_status_code

In [29]:
batch.head()

,case_id,task_id,vendor_id,vendor_address_id,eta,eta_extension,vendor_category,vendor_candidate_number,dispatch_response_description,dispatch_response_id,dispatch_response_reason,dispatch_type,task_status_code,enroute_miles,is_dispatch,service_time_eastern,service_time_utc,dispatch_request_time_eastern,dispatch_request_time_utc
0,2.263692e+09,1,25795,1,45.0,0.0,PA,1.0,ACCEPTED,2,Available,edispatch,DCM,12.25,True,2019-01-07 09:40:45,2019-01-07 14:40:45,2019-01-07 09:40:47,2019-01-07 14:40:47
1,2.263695e+09,1,46182,1,0.0,0.0,None,1.0,OPEN,1,Available,agent_call,ADV,0.00,False,2019-01-07 10:00:56,2019-01-07 15:00:56,2019-01-07 10:00:57,2019-01-07 15:00:57
2,2.263703e+09,1,30957,1,50.0,0.0,PA,1.0,CANCELED,4,Customer Cancel,edispatch,CCAN,3.14,True,2019-01-07 11:02:27,2019-01-07 16:02:27,2019-01-07 11:02:29,2019-01-07 16:02:29
3,2.263704e+09,1,112537,1,75.0,0.0,PA2,2.0,ACCEPTED,2,Available,edispatch,DCM,10.62,True,2019-01-07 11:01:58,2019-01-07 16:01:58,2019-01-07 11:02:53,2019-01-07 16:02:53
4,2.263368e+09,1,45979,1,40.0,0.0,PA,1.0,ACCEPTED,2,Available,agent_call,DCM,22.23,True,2019-01-07 11:11:26,2019-01-07 16:11:26,2019-01-07 11:02:59,2019-01-07 16:02:59


In [216]:
category_cols = ['vendor_category', 'dispatch_response_description', 'dispatch_type', 'task_status_code']

In [31]:
for col in category_cols:
    print(list(batch.get_column_value_counts(col, sort='value').keys()), '\n')

['AA', 'BST', 'D', 'DLROWN', 'DP', 'MT', 'PA', 'PA0', 'PA01', 'PA2', 'PA3', 'TP', 'UNLISTED SERVICE PROVIDER', 'UnListed Service Provider', 'Unknown', 'ZPB', 'ZPP', 'ZPS'] 

['ACCEPTED', 'CANCELED', 'OPEN', 'REFUSED'] 

['agent_call', 'dispatch_bot', 'edispatch'] 

['ADV', 'AGCN', 'CCAN', 'DA', 'DCM', 'DEC', 'DR', 'DS', 'HLD', 'IS', 'NCC', 'SCM', 'SPCN', 'SPI', 'SU', 'TCR'] 



In [32]:
print(batch.get_column_value_counts('dispatch_response_reason', sort='value'))

value
Available                                        1043473
Can Not Assist - Dealership Currently Closed           4
Can not cross state lines                            765
Can not transport customers                          809
Code Black                                         99760
Customer Cancel                                    59456
Customer Rejected                                      2
Customer Requested a Different Dealer                  2
Do not accept customers form of payment             7694
Do not have the proper equipment at this time      56966
ETA Issues                                         35328
ETA Issues-General                                 11976
ETA over 90 minutes                                    2
Equipment not in Fleet                             60098
Hold by Customer                                    1256
No longer offer that service                         573
Other                                             109443
Out of My Coverage Area  

In [33]:
vc_set = list(batch.get_column_value_counts('vendor_category', sort='value').keys())
drd_set = list(batch.get_column_value_counts('dispatch_response_description', sort='value').keys())
dt_set = list(batch.get_column_value_counts('dispatch_type', sort='value').keys())
tsc_set = list(batch.get_column_value_counts('task_status_code', sort='value').keys())

val_set_list = []

for col in category_cols:
    list_sets = list(batch.get_column_value_counts(col, sort='value').keys())
    val_set_list.append(list_sets)

In [34]:
for col, x in zip(category_cols,val_set_list):
    print(col, '\n', x, '\n')

vendor_category 
 ['AA', 'BST', 'D', 'DLROWN', 'DP', 'MT', 'PA', 'PA0', 'PA01', 'PA2', 'PA3', 'TP', 'UNLISTED SERVICE PROVIDER', 'UnListed Service Provider', 'Unknown', 'ZPB', 'ZPP', 'ZPS'] 

dispatch_response_description 
 ['ACCEPTED', 'CANCELED', 'OPEN', 'REFUSED'] 

dispatch_type 
 ['agent_call', 'dispatch_bot', 'edispatch'] 

task_status_code 
 ['ADV', 'AGCN', 'CCAN', 'DA', 'DCM', 'DEC', 'DR', 'DS', 'HLD', 'IS', 'NCC', 'SCM', 'SPCN', 'SPI', 'SU', 'TCR'] 



In [35]:
for col, vals in zip(category_cols, val_set_list):
    print(col, '\n', batch.expect_column_values_to_be_in_set(col, vals, mostly=0.8, result_format='BASIC', 
                                        include_config=True, catch_exceptions=True), '\n')
    # dispatch_response_description & dispatch_type are complete 
    # Lookup table for vendor_category & task_status_code values

vendor_category 
 {'success': True, 'result': {'element_count': 2202075, 'missing_count': 24130, 'missing_percent': 1.0957846576524415, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_in_set', 'kwargs': {'column': 'vendor_category', 'value_set': ['AA', 'BST', 'D', 'DLROWN', 'DP', 'MT', 'PA', 'PA0', 'PA01', 'PA2', 'PA3', 'TP', 'UNLISTED SERVICE PROVIDER', 'UnListed Service Provider', 'Unknown', 'ZPB', 'ZPP', 'ZPS'], 'mostly': 0.8, 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}} 

dispatch_response_description 
 {'success': True, 'result': {'element_count': 2202075, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_typ

### 6. Looking eta values to be in a range

In [36]:
batch.expect_column_values_to_be_between('eta', min_value=0, max_value=1000, mostly=0.90, result_format='BASIC', 
                                         catch_exceptions=True)

{'success': True,
 'result': {'element_count': 2202075,
  'missing_count': 775689,
  'missing_percent': 35.22536698341337,
  'unexpected_count': 333,
  'unexpected_percent': 0.0151221007458874,
  'unexpected_percent_nonmissing': 0.0233457142736959,
  'partial_unexpected_list': [1200.0,
   1660.0,
   4500.0,
   1200.0,
   1200.0,
   2250.0,
   4551.0,
   7070.0,
   4500.0,
   1120.0,
   1200.0,
   2100.0,
   3855.0,
   1303.0,
   2012.0,
   1200.0,
   2400.0,
   1250.0,
   7500.0,
   2222.0]},
 'exception_info': {'raised_exception': False,
  'exception_message': None,
  'exception_traceback': None}}

### 7. Looking dispatch_request_time to be greater than service_time

In [37]:
batch.expect_column_pair_values_A_to_be_greater_than_B('dispatch_request_time_utc', 'service_time_utc', mostly=0.7,
                                                      ignore_row_if='either_value_is_missing', result_format='BASIC',
                                                      catch_exceptions=True)

{'success': True,
 'result': {'element_count': 2202075,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 558414,
  'unexpected_percent': 25.35853683457648,
  'unexpected_percent_nonmissing': 25.35853683457648,
  'partial_unexpected_list': [['2019-01-07 16:02:59', '2019-01-07 16:11:26'],
   ['2019-01-07 17:44:18', '2019-01-07 18:23:23'],
   ['2019-01-07 22:21:27', '2019-01-07 22:27:31'],
   ['2019-01-07 15:01:15', '2019-01-07 15:04:05'],
   ['2019-01-07 18:26:48', '2019-01-07 18:33:21'],
   ['2019-01-07 19:38:05', '2019-01-07 19:46:30'],
   ['2019-01-07 21:22:50', '2019-01-07 21:58:11'],
   ['2019-01-08 00:12:33', '2019-01-08 02:19:38'],
   ['2019-01-07 11:30:24', '2019-01-07 12:30:00'],
   ['2019-01-07 15:24:03', '2019-01-07 15:24:03'],
   ['2019-01-07 19:17:13', '2019-01-07 19:55:47'],
   ['2019-01-07 22:26:15', '2019-01-07 23:02:45'],
   ['2019-01-07 23:34:28', '2019-01-08 00:15:23'],
   ['2019-01-07 12:50:50', '2019-01-07 12:58:21'],
   ['2019-01-07 16:26:33', '

### 8. Expecting columns to be certain data type

In [38]:
for x, y in batch.dtypes.iteritems():
    print(x, y)

case_id float64
task_id int64
vendor_id object
vendor_address_id int64
eta float64
eta_extension float64
vendor_category object
vendor_candidate_number float64
dispatch_response_description object
dispatch_response_id int64
dispatch_response_reason object
dispatch_type object
task_status_code object
enroute_miles float64
is_dispatch bool
service_time_eastern datetime64[ns]
service_time_utc datetime64[ns]
dispatch_request_time_eastern datetime64[ns]
dispatch_request_time_utc datetime64[ns]


In [39]:
for col, typ in batch.dtypes.items():
    print(batch.expect_column_values_to_be_of_type(col, str(typ), result_format='BASIC', catch_exceptions=True))

{'success': True, 'result': {'observed_value': 'float64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'int64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'object_'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'int64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'float64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'float64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'ob

### Review the expectations

Expectations that were true on this data batch were added. To view all the expectations you added so far about this data asset, do:

In [40]:
batch.get_expectation_suite()

2020-01-31T18:21:57-0500 - INFO - 	64 expectation(s) included in expectation_suite. result_format settings filtered.


{'data_asset_name': 'agero_dsa_pandas/default/network_outreach',
 'expectation_suite_name': 'warnings',
 'meta': {'great_expectations.__version__': '0.8.7'},
 'expectations': [{'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'case_id'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'task_id'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'vendor_id'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'vendor_address_id'}},
  {'expectation_type': 'expect_column_to_exist', 'kwargs': {'column': 'eta'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'eta_extension'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'vendor_category'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'vendor_candidate_number'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'dispatch_response_description

In [41]:
batch.save_expectation_suite()

2020-01-31T18:26:29-0500 - INFO - 	64 expectation(s) included in expectation_suite. result_format settings filtered.


### You created and saved expectations for at least one of the data assets.

### We will show you how to set up validation - the process of checking if new files of this type conform to your expectations before they are processed by your pipeline's code. 

### Go to [integrate_validation_into_pipeline.ipynb](integrate_validation_into_pipeline.ipynb) to proceed.


